In [ ]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import os

# Step 1: Taking Video Input

In [ ]:
camera = cv2.VideoCapture(0)

frame_count = camera.get(cv2.CAP_PROP_FRAME_COUNT)
fps = camera.get(cv2.CAP_PROP_FPS)

output = cv2.VideoWriter('./Video.avi', cv2.VideoWriter_fourcc(*'MJPG'), fps, (640, 480))

if not camera.isOpened():
    print("Error in opening camera")
    exit()

while True:
    frame_count += 1
    if int(frame_count//fps) >= 3:
        break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    ret, frame = camera.read()
    cv2.imshow('Capturing Video', frame)
    output.write(frame)

camera.release()
output.release()
cv2.destroyAllWindows()

# Step 2: Frame Selection

In [ ]:
video = cv2.VideoCapture('./Video.avi')
fps = video.get(cv2.CAP_PROP_FPS)
frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)

number_of_frames_required = 15
seconds = frame_count/(fps*number_of_frames_required)
print('video started')

for extracted_frame in range(number_of_frames_required):
    frame_id = int(fps*seconds*extracted_frame)
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
    ret, frame = video.read()
    cv2.waitKey(0)
    cv2.imwrite("./frames/" + str(extracted_frame) + ".png", frame)

# Step 3 and 4: Face Detection, Head Count and Image Refinement
### Implementation of Viola Jones Algorithm
This document contains the basic implementation of the Viola-Jones algorithm

In [ ]:
min_number_of_heads = 10000
max_number_of_heads = 0
cropped = 0

for frame_number in range(number_of_frames_required):
    frame = cv2.imread('./frames/' + str(frame_number) + '.png')
    grayscale_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml') 
    detected_faces = face_cascade.detectMultiScale(grayscale_image)
    min_number_of_heads = min(min_number_of_heads, len(detected_faces))
    max_number_of_heads = max(max_number_of_heads, len(detected_faces))
    for (column, row, width, height) in detected_faces:
        # cv2.rectangle(frame,(column, row),(column + width, row + height),(0, 255, 0),4)
        cv2.imwrite("./refined-images/" + str(cropped) + ".png", frame[row:row+height, column:column+width])
        cropped += 1
    # plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    # plt.axis("off")
    # plt.show()

In [ ]:
# HEAD COUNT
if min_number_of_heads == max_number_of_heads:
    print('Number of Students in the class are: ' + str(min_number_of_heads))
else:
    print("Number of Students in the class are within the range: " + str(min_number_of_heads) + '-' + str(max_number_of_heads))

# Step 5: Face Recognition

In [ ]:
from deepface import DeepFace

In [ ]:
# dfs = DeepFace.find(img_path = "face1.png", db_path = "./database/")
# print(dfs)

In [ ]:
# result = DeepFace.verify(img1_path = "face1.png", img2_path = "./database/0.png")
# print(result)

In [ ]:
# for cropped_img in range(cropped):
#     try:
#         result = DeepFace.verify(img1_path = "./refined-images/" + str(cropped_img) + ".png", img2_path = "./database/0.png")
#     except:
#         result = "false"
#     print(result)

In [ ]:
present = []

for cropped_img in range(cropped):
    for filename in os.listdir("./tmp"):
        rollnumber = str(filename).split(".")[0]
        if(rollnumber not in present):
            try:
                result = DeepFace.verify(img1_path = "./refined-images/" + str(cropped_img) + ".png", img2_path = "./tmp/" + str(filename))
                if (result['verified'] == True):
                    present.append(rollnumber)
            except:
                result = False

print("Attendance is: ")
print(present)

# Final Step: Accepting Class Details and Posting data to Firebase Realtime Database

In [ ]:
# Accepting Class details:
import datetime

year = str(input("Enter year: "))
stream = "CSE"
course = str(input("Enter course (BTech or MTech)"))
subject = str(input("Enter Subject Code"))
timestamp = datetime.datetime.now()

In [ ]:
import firebase_admin
from firebase_admin import db

cred_obj = firebase_admin.credentials.Certificate('secret key.json')
default_app = firebase_admin.initialize_app(cred_obj, {
	'databaseURL': "https://smartattend-6da73-default-rtdb.firebaseio.com/"
})

# Get a database reference to our posts
ref = db.reference('Attendance')

In [ ]:
# # Posting Data to Firebase Realtime database
# import requests
# path = f'https://smartattend-6da73-default-rtdb.firebaseio.com/Attendance/{course}/{stream}/{year}/{subject}'
# # print(path)
# requests.post(path, data='{"words/banana":true, "words/pie":true}')

In [ ]:
attendance_ref = ref.child(f'{course}/{stream}/{year}/{subject}/')

# Updating the number of classes held
try:
    totalClassesHeld = int(ref.get()[course][stream][year][subject]['totalClassesHeld'])+1
    print(totalClassesHeld)
    attendance_ref.update({
        'totalClassesHeld': str(totalClassesHeld)
    })
except:
    attendance_ref.update({
        'totalClassesHeld': 1
    })

In [ ]:
for student in present:
    attendance_ref.push().set({
        'enrollmentNumber': student,
        'timestamp': str(timestamp)
})